In [78]:
# Things to Fix

# datapoints should not include flow, head as x,y coordinates


In [143]:
# IMPORTS
import xml.etree.ElementTree as ET
import os
import pandas as pd
import numpy as np

In [80]:
# Pointing to curve PSD
psd_path = r"C:\Users\104092\OneDrive - Grundfos\Documents\1 - PROJECTS\SPE Integration\SPE_PumpCurve"
psd_file = r"Populated Curve PSD - 2024 curve testing.xlsx"
psd_filepath = os.path.join(psd_path, psd_file)

In [193]:
# This is the folder/file with the curve export csv
myDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\1 - PROJECTS\SPE Integration\SPE_PumpCurve"
myFile = "PumpCurves.csv"
filePath = os.path.join(myDir, myFile)

# This creates a dataframe of the curve export csv, and fills in the RPM(curve nominal) column
data = pd.read_csv(filePath, sep=";", index_col=False, skip_blank_lines=False)
data = data[data['ProductNumber'].notna()]
data['ProductNumber_str'] = data['ProductNumber'].astype(int).astype(str)
# data.dtypes

### Functions

In [82]:
def add_namespace(elem_tag, xsi_namespace):
    """Adds namespace to root node."""
    XHTML_NAMESPACE = xsi_namespace
    XHTML = "{%s}" % XHTML_NAMESPACE
    NSMAP = {'xsi' : XHTML_NAMESPACE} # the default namespace (no prefix)

    return ET.Element(elem_tag, nsmap=NSMAP) # lxml only!

In [83]:
def add_elem_from_dict(parent_elem, elem_dict={'':''}):
    """Takes elements inside elem_dict and adds as elements to parent_elem"""
    if elem_dict:
        for key, value in elem_dict.items():
            elem = ET.SubElement(parent_elem, key)
            elem.text = str(value)
    else:
        elem = ET.SubElement(parent_elem)

In [84]:
def create_pump_curve_dict(row) -> dict:
    """Returns dictionary of updated attributes to be converted to elements """
	
    pumpCurve_dict = {
        'curveNumber': row['Curve number'],
        'speedRef': row['Speed, data'],
        # 'polesRef': row['Poles'],
        'hzRef': row['Hz'],
        # 'mcsfMinRef': row['MCSF @ min impeller diameter'],
        # 'mcsfMaxRef': row['MCSF @ max impeller diameter'],
        'eyeCount': 1,
        'speedCurveNominal': row['Speed, nominal'],
        'speedCurveMin': row['Speed, Min'],
        'speedCurveMax': row['Speed, max'],
        'diaImpInc': row['Diameter increment'],
        # 'speedVariableCurveMin': row['Variable speed min limit'],
		'speedVariableCurveMax': row['Speed, max'],
        # 'optionalCurveType': 'Power',
		'optionalCurveType': 'Efficiency',
        'flowStartHeadEnabled': 'false',
        'flowStartEtaEnabled': 'false',
        'flowStartPowerEnabled': 'false',
		'flowStartNPSHEnabled':'false',
		'flowStopNPSHEnabled':'false',
		'flowStartSubmergenceEnabled':'false',
		'extendNpshToMcsfMin':'false',
		'catalogTrimsSelectionMode':'0',
		'styleCurveBelowStart':'none',
		'flowExponentTrim':'1.0',
		'headExponentTrim':'2.0',
		'npshExponentTrim':'0.0',
		'etaExponentTrim':'0.0',
		'powerDriverFixed':'0.0',
		'quantityMotors':'1',
		'serviceFactorDriverFixed':'1.0',
		'serviceFactorDriverFixedUsed':'false',
		'flowExponentSpeed':'1.0',
		'headExponentSpeed':'2.0',
		'etaExponentSpeedReduced':'0.0',
		'etaExponentSpeedIncreased':'0.0',
		'npshExponentSpeedReduced':'2.0',
		'npshExponentSpeedIncreased':'2.0',
		'submergenceExponentSpeedReduced':'2.0',
		'submergenceExponentSpeedIncreased':'2.0',
		'hideEfficiencyInSelector':'false',
		'speedOfSoundRef':'331.6583',
		'speedOfSoundExpFlow':'1.0',
		'speedOfSoundExpHead':'2.0',
		'speedOfSoundExpEta':'0.0',
		'speedOfSoundExpEtaTotal':'0.0',
		'temperatureGasInletSkb':'20.0',
		'pressureGasInletSkb':'1.01325',
		'relativeHumidityGasSkb':'50.0',
		'diaRotatingElement':'0.0',
		'solveVariantDisplayStrategy':'2',
		'flowStopPercentBEP':'0.0',
		'headMarginFixedDia':'0.0',
		'headMarginFixedDiaPercentage':'0.0',
		'submergenceVortexMin':'0.0',
		'submergenceStartupMin':'0.0',
		'thrustFactor':'0.0',
		'thrustFactorBalanced':'0.0',
		'displayBothDiameters':'false',
		# 'isoEfficiencyValues':'56:62:65:68',
		'moiFirstStage':'0.0',
		'moiAdditionalStage':'0.0',
		'moiPumpCoupling':'0.0',
		'flowMaxAllowedMinRef':'0.0',
		'flowMaxAllowedMaxRef':'0.0',
		'loadRadialRef':'0.0'
		}

    return pumpCurve_dict

In [206]:
def get_impeller_trim(curve_number:str):

    # Filter the DataFrame based on a condition in 'column1'
    condition = data['ProductNumber_str'] == curve_number
    # result = df.loc[condition, 'Impeller size']
    result = data.loc[condition, 'RPM(Curve nominal)'] 
    result = result.to_list()

    return(result[0])

In [207]:
def create_impeller_dict(curve_number:str) -> dict:
# def create_impeller_dict(df, new_curve_number:str, fixed_speed) -> dict:
	"""Creates dict of attributes to add to each Impeller node. """
	
	# append_dict contains appropriate values for attributes/tags in updates_list
	# append_dict = copy_values_from_source_xml(updates_list, new_curve_number)
	speed_or_trim = get_impeller_trim(curve_number)

	# If any of the below are important, add tag to updates_list
	impeller_dict = {
	# 'diameter': get_impeller_trim(df, new_curve_number),
	'diameter': speed_or_trim,
	'flowStartNPSH':'0.0,',
	'diameterHubSide':'0.0',
	'weight':'0.0',
	'surgeFlow':'0.0',
	'flowStartEta':'0.0',
	'flowStartHead':'0.0',
	'flowStartNPSH':'0.0',
	'flowStartNPSH0Percent':'0.0',
	'flowStartNPSHIncipient':'0.0',
	'flowStopNPSH':'0.0',
	'flowStopNPSH0Percent':'0.0',
	'flowStopNPSHIncipient':'0.0',
	'flowStartSubmergence':'0.0',
	'flowStartPower':'0.0',
	'powerShutoffFixedEnabled':'false',
	'powerShutoffFixed':'0.0',
	'bepFixedEnabled':'true',
	'solveVariantMin':'0.0',
	'solveVariantMax':'0.0',
	'minimumVolumetricEfficiency':'0.0',
	'minimumVolumetricEfficiencyRated':'0.0',
	'maximumDifferentialPressure':'0.0',
	'stopFlow':'0.0'
	}
	
	# append_dict.update(impeller_dict)
	# print(f"final dict to be added: {append_dict}")
	
	# return(append_dict)
	return(impeller_dict)

In [87]:
def add_curve_data_points(parent_elem, curve_number, curve_type):
    """ Adds curve data points for flow/power, flow/head, flow/NPSH """

    # Opens relevant curve tab in PSD, and grabs flow, head, power, npsh columns
    # curve_data_df = pd.read_excel(psd_filepath,sheet_name=curve_number, header=7, skiprows=[8], usecols="D,E,L,S", nrows=50)
    curve_data_df = pd.read_excel(psd_filepath,sheet_name=curve_number, names=['Flow','Head','Eta','NPSH'], skiprows=[8], usecols="D,E,L,S", nrows=50)
    curve_data_df = curve_data_df.dropna()

    # print(f'curvenumber type: {type(curve_number)}, column headers: {list(curve_data_df.columns)}')

    # Iterate through curve data df and create dicts of each data point that will be added as nodes to output xml
    for index, row in curve_data_df.iterrows():
        datapoint_elem = ET.SubElement(parent_elem, "DataPoint", disabled="false")
        
        if curve_type == 'Efficiency':
            datapoint_dict = {
                # 'x': metric_to_us(row['Flow'], "flow"),
                # 'y': metric_to_us(row[curve_type], 'power'),
                'x': row['Flow'],
                'y': row['Eta'],
                'isOnCurve':'false',
                'division':'false',
                'useCubicSplines':'false',
                'slopeEnabled':'false'
            }
        
        elif curve_type == 'Power':
            datapoint_dict = {
                # 'x': metric_to_us(row['Flow'], "flow"),
                # 'y': metric_to_us(row[curve_type], 'power'),
                'x': row['Flow'],
                'y': row[curve_type],
                'isOnCurve':'false',
                'division':'false',
                'useCubicSplines':'false',
                'slopeEnabled':'false'
            }

        elif (curve_type == 'Head'):
            datapoint_dict = {
                # 'x': metric_to_us(row['Flow'], "flow"),
                # 'y': metric_to_us(row[curve_type], 'distance'),
                'x': row['Flow'],
                'y': row[curve_type],
                'isOnCurve':'false',
                'division':'false',
                'useCubicSplines':'true',
                'slopeEnabled':'false'
            }

        elif (curve_type == 'NPSH'):
            datapoint_dict = {
                # 'x': metric_to_us(row['Flow'], "flow"),
                # 'y': metric_to_us(row[curve_type], 'distance'),
                'x': row['Flow'],
                'y': row[curve_type],
                'isOnCurve':'false',
                'division':'false',
                'useCubicSplines':'false',
                'slopeEnabled':'false'
            }

        else:
            print(f'curve_type not allowed: {curve_type}')
            
        add_elem_from_dict(datapoint_elem, datapoint_dict)

In [88]:
def add_curve(parent_elem, curve_type:str, curve_number):
    """ Creates <Curve> parent element, and adds specified curve to xml """
    curve_elem = ET.SubElement(parent_elem, 'Curve', type=curve_type)

    # Add Curve Data Points to Curve Element
    add_curve_data_points(curve_elem, curve_number, curve_type)

### Main

In [90]:
# Create Root Tag using Custom Fields
root_ns = "http://www.w3.org/2001/XMLSchema-instance"
curve_family_name = "SPE"

root = add_namespace('SKBData', root_ns)

In [91]:
# Add <CurveFamily> node
curveFamily_elem = ET.SubElement(root, "CurveFamily")

# Add tag For type of trim curves: "speed" or "impellerDiamter"
# svData_type_tag = ET.SubElement(curveFamily_elem, "svDataType")
# interpData_type_tag = ET.SubElement(curveFamily_elem, "interpDataType")

header_dict = {
        'name': curve_family_name,
#         'impellerTyp':'radialFlow',
        # 'svDataType':'impellerDiamter',
        # 'interpDataType':'impellerDiamter',
        'svDataType':'speed',
        'interpDataType':'speed',
        'compressorConditionsInputTypeSkb':'speedOfSound',
        'flowTypeSkb':'volumetricFlow',
        'headTypeSkb':'head',
        'headMarginForFixedDiameter':'value',
        'submergenceMethod':'fixedValue',
        'errorFitMax':'1.5',
        'pumpType':'0',
        'interpQty':'4',
        'efficiencyPowerDataType':'pump'
}

add_elem_from_dict(curveFamily_elem, header_dict)

In [94]:
# Add pump curve collection. do for each curve tab
curve_header_data = pd.read_excel(psd_filepath,sheet_name="Curve Header Data", header=8, skiprows=[9], converters={'Curve number':str})
qname = ET.QName(root_ns,"type")

In [208]:
# Need to set appropriate values before adding to XML tree. Will need to iterate through curve_header_data to update values.
for index, row in curve_header_data.iterrows():
    # <pumpCurveCollection xsi:type="CentrifugalPumpCurveCollection"> This is the parent of each pump curve"
    pumpCurveCollection_elem = ET.SubElement(curveFamily_elem, 'pumpCurveCollection', {qname: "CentrifugalPumpCurveCollection"})

    # Creates pump curve elements
    curve_dict = create_pump_curve_dict(row)
    add_elem_from_dict(pumpCurveCollection_elem, curve_dict)

    # Add Impeller Elements
    impeller_elem = ET.SubElement(pumpCurveCollection_elem, 'Impeller')
    
    # Here we are trying to current impeller trim/vfd speeds from original GPI curve export csv
    # impeller_dict = create_impeller_dict(data, row['Curve number'], row['Speed, data'])
    impeller_dict = create_impeller_dict(row['Curve number'])
    add_elem_from_dict(impeller_elem, impeller_dict)

    # Add Curve Elements
    add_curve(impeller_elem, "Head", row['Curve number'])
    # add_curve(impeller_elem, "Power", row['Curve number'])
    add_curve(impeller_elem, "Efficiency", row['Curve number'])
    add_curve(impeller_elem, "NPSH", row['Curve number'])

### Print

In [209]:
tree = ET.ElementTree(root)
tree.write("filename.xml")

In [210]:
import xml.dom.minidom

with open('filename.xml') as OriginalXML:
    temp = xml.dom.minidom.parseString(OriginalXML.read())
    New_XML = temp.toprettyxml()
  
print(New_XML)

<?xml version="1.0" ?>
<SKBData xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" nsmap="{'xsi': 'http://www.w3.org/2001/XMLSchema-instance'}">
	<CurveFamily>
		<name>SPE</name>
		<svDataType>speed</svDataType>
		<interpDataType>speed</interpDataType>
		<compressorConditionsInputTypeSkb>speedOfSound</compressorConditionsInputTypeSkb>
		<flowTypeSkb>volumetricFlow</flowTypeSkb>
		<headTypeSkb>head</headTypeSkb>
		<headMarginForFixedDiameter>value</headMarginForFixedDiameter>
		<submergenceMethod>fixedValue</submergenceMethod>
		<errorFitMax>1.5</errorFitMax>
		<pumpType>0</pumpType>
		<interpQty>4</interpQty>
		<efficiencyPowerDataType>pump</efficiencyPowerDataType>
		<pumpCurveCollection xsi:type="CentrifugalPumpCurveCollection">
			<curveNumber>92673182</curveNumber>
			<speedRef>3600</speedRef>
			<hzRef>60</hzRef>
			<eyeCount>1</eyeCount>
			<speedCurveNominal>3600</speedCurveNominal>
			<speedCurveMin>100</speedCurveMin>
			<speedCurveMax>3600</speedCurveMax>
			<diaImpInc>0.0

: 